In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# since keras_vggface is not available on Kaggle, install it directly from Github
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import keras
import keras_vggface
from tqdm import tqdm
from scipy.spatial import distance
import random

In [ ]:
# read pairings data
train_df = pd.read_csv("../input/recognizing-faces-in-the-wild/train_relationships.csv")

In [ ]:
train_df.head()

In [ ]:
def preprocess_image(img, image_size):
    '''
    Take a PIL.Image object, resize to have shape (image_size, image_size), convert to grayscale and reduce
    pixels' values to the range [0,1]. Returns a numpy ndarray.
    '''
    return np.asarray(img.resize((image_size, image_size), PIL.Image.ANTIALIAS).convert('L')) / 255.0

def preprocess_facenet(img, axis=-1):
    '''
    Preprocess images as required by Facenet. In particular, resize to 160x160x3.
    '''
    return np.asarray(img.resize((160, 160), PIL.Image.ANTIALIAS)) / 255.0
    
def preprocess_vgg_face(img, version=2):
    '''
    Preprocess an image 'img' as required by VGGFace. In particular, call the dedicated function with
    the correct version (version=2 is for 'resnet' implementation of the net).
    '''
    return keras_vggface.utils.preprocess_input(np.asarray(img).astype(np.float), version=version)

def l2_normalization(image, epsilon=1e-10):
    '''
    Performs L2 normalization on a set of input images 'image'. Namely, we divide by the square root
    of the squared image, while taking the maximum with a small 'epsilon' ensures we avoid division by 0.
    '''
    return image / np.sqrt(np.maximum(np.sum(np.square(image), axis=-1, keepdims=True), epsilon))

def l2(x, y):
    '''
    Return the l2-norm distance among images x and y.
    '''
    return np.linalg.norm(x - y)

In [ ]:
def load_train_metadata(base_path):
    # create a pandas DataFrame having the following columns. It will store useful information about each image
    data = pd.DataFrame(columns=["ID", "FamilyID", "PersonID", "PictureID", "Image"])
    num = 0
    # iterate over directory tree, and add each picture (preprocessed) and its metadata to the dataframe
    for family in os.listdir(base_path):
        for person in os.listdir(base_path + family):
            image_num = 0  # the number of pictures for person. We need this because the numbering in the dataset is sort of odd
            # there are some empty folders in the training set. This loop never executes if we encounter one of them
            for picture in os.listdir(base_path + family + "/" + person):
                # add a new entry to the dataframe
                data.loc[num] = pd.Series({"ID": family + "/" + person, "FamilyID": family, "PersonID": person, "PictureID": image_num, "Image": num})          
                image_num += 1
                num += 1
    return data

In [ ]:
def load_train_images(base_path, preprocessor, num_images=12379, image_size=224):
    # create a pandas DataFrame having the following columns. It will store useful information about each image
    data = pd.DataFrame(columns=["ID", "FamilyID", "PersonID", "PictureID", "Image"])
    images = np.zeros((num_images, image_size, image_size, 3)) # pre-allocate array of images
    num = 0
    # iterate over directory tree, and add each picture (preprocessed) and its metadata to the dataframe
    for family in os.listdir(base_path):
        for person in os.listdir(base_path + family):
            image_num = 0  # the number of pictures for person. We need this because the numbering in the dataset is sort of odd
            # there are some empty folders in the training set. This loop never executes if we encounter one of them
            for picture in os.listdir(base_path + family + "/" + person):
                path = base_path + family + "/" + person + "/" + picture
                image = PIL.Image.open(path)
                # add a new entry to the dataframe
                data.loc[num] = pd.Series({"ID": family + "/" + person, "FamilyID": family, "PersonID": person, "PictureID": image_num, "Image": num})          
                # insert a new image in the array, after preprocessing
                images[num, :, :, :] = preprocessor(image)
                image_num += 1
                num += 1
    return data, images

In [ ]:
def load_test_images(base_path, image_size, num_images=6282):
    # pre-allocate array of test images
    images = np.zeros((num_images, image_size, image_size, 3))
    num = 0
    # open and insert in the array one preprocessed image after the other
    for person in os.listdir(base_path):
        images[num, :, :, :] = preprocess_facenet(PIL.Image.open(base_path + person))
        num += 1
    return images

In [ ]:
train_data = load_train_metadata("../input/recognizing-faces-in-the-wild/train/")

In [ ]:
#_, train_images = load_train_images("../input/recognizing-faces-in-the-wild/train/", preprocess_vgg_face)

In [ ]:
train_data = train_data.set_index("ID") # computationally efficient for the following task
train_data.head()

In [ ]:
# build a dictionary mapping each unique person ID to the list of kins, as given by train_df
#relations = dict()
#for pair in train_df.index:
#    elem = train_df.loc[pair, "p1"]
#    if elem in relations:
#        relations[elem].append(train_df.loc[pair, "p2"])
#    else:
#        relations[elem] = [train_df.loc[pair, "p2"]]

# create a new column in the main dataframe for the kins. Notice some persons will have NaN
#train_data["Relations"] = pd.Series(relations)
#del relations

In [ ]:
# retrieve the number of unique families
len(train_data["FamilyID"].unique())

In [ ]:
# retrieve the total number of pictures
train_data.shape[0]

In [ ]:
# retrieve all unique persons
len(train_data.index.unique())

In [ ]:
# load the Facenet model for Keras, as implemented at https://github.com/nyoki-mtl/keras-facenet
#model = keras.models.load_model("../input/facenet-keras/facenet_keras.h5")

In [ ]:
def embed_facenet(model, images, batch_size):
    '''
    Compute predictions (embeddings) for a set of images 'images', assuming 'model' is the Facenet
    model that can be found at https://github.com/nyoki-mtl/keras-facenet. Predictions is unrolled
    into batches of size 'batch_size'. Choosing a batch size too large will likely cause the kernel 
    to die.
    '''
    embedding_size = 128 # hand-recovered size of the Facenet output
    num_images = np.shape(images)[0]
    # pre-allocate array of image embeddings
    embeddings = np.zeros((num_images, embedding_size))
    start = 0
    stop = batch_size
    # loop over the batches and process them, using 'start' and 'stop' as offsets
    while stop <= num_images:
        print("Processing next batch...")
        embeddings[start:stop, :] = model.predict_on_batch(images[start:stop, :, :, :])
        start += batch_size
        stop += batch_size
    # we still miss the last (possibly, incomplete) batch
    embeddings[start:, :] = model.predict_on_batch(images[start:, :, :, :])
    return embeddings

In [ ]:
#help(keras_vggface.vggface.VGGFace)

In [ ]:
###
### THERE SEEM TO BE SPURIOUS ENTRIES IN TRAIN_DF, THAT IS PAIRS WITH NON-EXISTENT PEOPLE
###

In [ ]:
# condense in a list all the pairs contained in train_df. Take into account that not all paths appearing
# in train_df actually correspond to some training image
pairs = [(train_df.loc[idx, "p1"], train_df.loc[idx, "p2"]) for idx in train_df.index 
         if train_df.loc[idx, "p1"] in train_data.index and train_df.loc[idx, "p2"] in train_data.index]

In [ ]:
def get_train_image(image_path, preprocessor):
    '''
    Return a Numpy array of the image found at 'image_path' among the training images. Use
    the 'preprocessor' to preprocess the image as required. If multiple photos for the same
    individual are found, return a randomly picked one.
    '''
    base_path = "../input/recognizing-faces-in-the-wild/train/" + image_path
    image = PIL.Image.open(base_path + "/" + random.choice(os.listdir(base_path)))
    return preprocessor(image)

In [ ]:
def sample_pairs(num, pairs, data):
    '''
    Return a sample of paired images. The sampling is done in such a way that positive and negative
    pairs are in equal proportion. This function requires the number of couples to return, a ground-truth
    list for the positive pairs and the images metadata.
    '''
    couples = list() # the pairs sampled so far
    # sample num//2 positive pairs
    while len(couples) < num//2:
        couple = random.sample(pairs, k=1)[0] # sample a pair
        couples.append(couple)
    # sample (num - num//2) negative pairs 
    while len(couples) < num:
        img1, img2 = random.sample(list(data.index), 2) # sample two images from the general set
        # add them only if they are no-kin
        if (img1, img2) not in pairs and (img2, img1) not in pairs:
            couples.append((img1, img2))
    return couples

In [ ]:
def compute_distances(couples, data, predictor, distance, preprocessor):
    '''
    Compute a distance metric among embeddings generated from a sample of images. Specifically,
    this function returns a Pandas DataFrame with a 'distance' column for the metric computed on
    the random pairs, and a 'label' column filled with 0 (no-kin) or 1 (kin). This function requires the
    pairs to sample, the metadata of the images, the predictor function of a model to use as feature 
    extractor, a distance function to be computed and a preprocessor to be applied on the images (since they
    are opened and preprocessed "on-the-fly".
    '''
    distances = list()
    num = len(couples)
    labels = [1 if idx <= num//2 else 0 for idx in range(num)] # we already know we will sample the first num//2 from the positives
    # loop over the pairs of images and compute the distance on their embeddings
    for img1, img2 in tqdm(couples, total=num):
        # compute embeddings. Of course, since some images might appear in more than one pair, we are doing
        # unnecessary work by loading and recomputing everytime their embeddings. Nevertheless, in order to 
        # avoid it we would need to allocate a quite large buffer of memory, clinching from a compute-bound 
        # workload to a memory-bound one, which is what we want to avoid. We also need to take into account
        # that an image is actually very unlikely to appear more than once.
        image1 = get_train_image(img1, preprocessor)
        image2 = get_train_image(img2, preprocessor)
        embeds = predictor(np.array([image1, image2]))
        # compute the distance
        distances.append(distance(embeds[0, :], embeds[1, :]))
    return pd.DataFrame({"distance": distances, "label": labels})

In [ ]:
# load the VGGFace model
vgg = keras_vggface.VGGFace(include_top=False, model="resnet50")

In [ ]:
# freeze the layers
for layer in vgg.layers:
    layer.trainable = True
for layer in vgg.layers[:-3]:
    layer.trainable = True

In [ ]:
#dist_df = compute_distances(2000, pairs, train_images, train_data, model, distance.euclidean)

In [ ]:
from matplotlib import style

style.use("ggplot")
sample_size = 1000

distances = {"euclidean": distance.euclidean, "cosine": distance.cosine, "l2": l2}
f, axes = plt.subplots(nrows=1, ncols=len(distances), figsize=(20,5))
f.suptitle("distances among image pairs", fontsize=20)
i = 0
# sample pairs, to be reused for each distance metric for consistency
sample = sample_pairs(sample_size, pairs, train_data)
# for each distance in 'distances', compute it among the pairs and plot a scatterplot
for ax in axes.flatten():
    dist = compute_distances(sample, train_data, model.predict_on_batch, distances[list(distances.keys())[i]], preprocess_facenet)
    # shuffle the dataframe, otherwise it is perfectly split between kin and no-kin
    dist = dist.sample(frac=1)
    ax.set_title(list(distances.keys())[i])
    plot = sns.scatterplot(list(range(len(dist))), dist["distance"], hue=dist["label"], ax=ax)
    # plot a horizontal line at an 'ideal' boundary, handcrafted
    i += 1
del dist
del sample

In [ ]:
# sample a given number of families to be used as hold-out validation set
#val = np.random.choice(train_data["FamilyID"], size=20)

In [ ]:
# include in the validation set all the pairs corresponding to the sampled families, making sure the individuals
# appear among the training images
#val = [(train_df.loc[idx, "p1"], train_df.loc[idx, "p2"]) for idx in train_df.index 
#       if str(train_df.loc[idx, "p1"]).split("/")[0] in val and str(train_df.loc[idx, "p2"]).split("/")[0] in val
#      and train_df.loc[idx, "p1"] in train_data.index and train_df.loc[idx, "p2"] in train_data.index]

In [ ]:
# update the list of training pairs removing those destined to the validation set
#pairs = [couple for couple in pairs if str(train_data.loc[couple[0], "FamilyID"]) not in val and 
#         str(train_data.loc[couple[1], "FamilyID"]) not in val] 

In [ ]:
def fakeGen(pairs, batch_size, data, preprocessor=preprocess_vgg_face):
    '''
    Temporary generator function. Works like 'sample_pairs', but the output conforms to the
    Keras 'fit_generator' routine for a siamese network, which requires a couple of inputs and
    their label.
    '''
    a = list() # the pairs sampled so far
    b = list()
    y = list()
    # sample num//2 positive pairs
    while len(a) < batch_size//2:
        couple = random.sample(pairs, k=1)[0] # sample a pair
        a.append(get_train_image(couple[0], preprocessor))
        b.append(get_train_image(couple[1], preprocessor))
        y.append(1)
    # sample (num - num//2) negative pairs 
    while len(a) < batch_size:
        img1, img2 = random.sample(list(data.index), 2) # sample two images from the general set
        # add them only if they are no-kin
        if (img1, img2) not in pairs and (img2, img1) not in pairs:
            a.append(get_train_image(img1, preprocessor))
            b.append(get_train_image(img2, preprocessor))
            y.append(0)
    # convert from list to ndarray
    a = np.array(a)
    b = np.array(b)
    y = np.array(y)
    # shuffle the three arrays. To make sure the relative order of the pairs and labels is kept,
    # we retrieve the current random number generator and reset it everytime. If we did not shuffle
    # the batches, the network would always see batch_size//2 positive pairs and then (batch_size - 
    # batch_size//2) negative pairs, making the learning sort of rough
    rng = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng)
    np.random.shuffle(b)
    np.random.set_state(rng)
    np.random.shuffle(y)
    return [a, b], y  

In [ ]:
def fakefakeGen(pairs, batch_size, data):
    while True:
        yield fakeGen(pairs, batch_size, data)

In [ ]:
#help(keras.Model.fit_generator)

In [ ]:
def initialize_bias(shape):
    return np.random.normal(loc=0.5, scale=1e-2,size=shape)
  
def initialize_weights(shape):
    return np.random.normal(loc=0.0, scale=1e-2, size=shape)

In [ ]:
val = [pair for pair in pairs if "F09" in pair[0]]
pairs = [pair for pair in pairs if "F09" not in pair[0]]

In [ ]:
len(val), len(pairs)

In [ ]:
from keras.layers import Input, Dense, Dropout, Concatenate, GlobalMaxPool2D, GlobalAvgPool2D, Multiply, Lambda
from keras import Model
from keras.optimizers import Adam
from keras.regularizers import l2
import keras.backend as K
import threading

dimension = (224, 224, 3)
batch_size = 32
epochs = 35

left_input = Input(dimension)
right_input = Input(dimension)

x1 = vgg(left_input)
x2 = vgg(right_input)
x3 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
x4 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])
fc = Dense(100, activation="relu", kernel_regularizer=l2(1e-3),kernel_initializer=initialize_weights,
           bias_initializer=initialize_bias)
x5 = fc(x3)
x6 = fc(x4)
x7 = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([x5, x6])
# |h1-h2|^2
x8 = Lambda(lambda tensor: K.square(tensor))(x7)
# h1*h2
x9 = Multiply()([x5, x6])
# |h1-h2|^2 + h1*h2
x10 = Concatenate(axis=-1)([x8,x9])

x11 = Dense(100,activation='relu',kernel_regularizer=l2(1e-3), kernel_initializer=initialize_weights,
            bias_initializer=initialize_bias)(x10)
x12 = Dropout(0.2)(x11)

prediction = Dense(1, activation="sigmoid", bias_initializer=initialize_bias)(x10)
siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)

# decreasing the learning rate by a factor of 100 reduced a lot the loss on the very first batch. Alos choosing a more suitable
# weight initialization seemed to produce benefits
siamese_net.compile(optimizer=Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])
siamese_net.fit_generator(fakefakeGen(pairs, batch_size, train_data), epochs=epochs, steps_per_epoch=(len(pairs) * 2)/batch_size, 
                          validation_steps=(len(val) * 2)/batch_size, validation_data=fakefakeGen(val, batch_size, train_data), 
                          use_multiprocessing=True)

In [ ]:
# The two following cells owe a lot to 'adityajn' user. Thanks a lot!

submission = pd.read_csv('../input/recognizing-faces-in-the-wild/sample_submission.csv')
submission['p1'] = submission.img_pair.apply( lambda x: '../input/test/'+x.split('-')[0] )
submission['p2'] = submission.img_pair.apply( lambda x: '../input/test/'+x.split('-')[1] )
print(submission.shape)
submission.head()

In [ ]:
probs = list()
for i,j in tqdm([ (0,500),(500,1000),(1000,1500),(1500,2000),(2000,2500),
                 (2500,3000),(3000,3500),(3500,4000),(4000,4500),(4500,5000),(5000,5310) ]):
    imgs1 = np.array( [ read_img(photo) for photo in submission.p1.values[i:j] ] )
    imgs2 = np.array( [ read_img(photo) for photo in submission.p2.values[i:j] ] )
    prob =  siamese_net.predict( [ imgs1, imgs2 ] )
    probs.append(np.squeeze(prob))
    del imgs1,imgs2
    gc.collect()

submission.is_related = np.concatenate(probs)
submission.drop( ['p1','p2'],axis=1,inplace=True )
submission.head()
submission.to_csv('submission.csv',index=False)

In [ ]:
#test_im = os.listdir("../input/recognizing-faces-in-the-wild/test/")
#test_df = pd.read_csv("../input/recognizing-faces-in-the-wild/sample_submission.csv")
#test_df["distance"] = 0
#img2idx = dict()
#for idx, img in enumerate(test_im):
#    img2idx[img] = idx
#for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
#    imgs = [norm[img2idx[img]] for img in row.img_pair.split("-")]
#    test_df.loc[idx, "distance"] = distance.euclidean(*imgs)
#all_distances = test_df.distance.values
#sum_dist = np.sum(all_distances)
#probs = []
#for dist in tqdm(all_distances):
#    prob = np.sum(all_distances[np.where(all_distances <= dist)[0]]) / sum_dist
#    probs.append(1 - prob)
#sub_df = pd.read_csv("../input/recognizing-faces-in-the-wild/sample_submission.csv").copy()
#sub_df.is_related = probs
#sub_df.to_csv("../input/submission.csv", index=False)

In [ ]:
#sub_df.head(100)

In [ ]:
#test = np.load("train_embeddings.npy")

In [ ]:
def cross_validation(data, pairs, num_folds):
    '''
    Perform the cross-validation strategy as in "Visual Kinship Recognition of Families in the Wild" (Robinson, 2017). 
    Given the training dataset 'data', and the dataframe of pairings 'pairs', split the former into
    num_folds foldings. Returns the indices of the foldings in a dictionary format, for example:
    {0: [1, 23, 5, 2], 1: [7, 9, 0], ...}; basically, the keys should be the fold numbers, while the values
    should be a list (or ndarray) of the corresponding indexes (to be used on the training dataset). 
    '''
    ans = dict()
    ## TODO: define the cross-validation folds ##
    # make sure each family does not overlap over folds, i.e. each family appears entirely in one and only one fold
    # randomly mismatch pairs in each fold until the number of negative and positive pairs are the same in each fold
    # (i.e., negative pairs are added at random until it makes up 50% of the respective fold). Thus, the total number 
    # of positive and negative labels are equivalent.
    return ans

In [ ]:
# One kernel generates an embedding for each image using a modified version of VGG, sort of VGG-Facenet, and then computes
# the cosine similarity and compresses everything to output a probability of being of the same family. What about computational
# budget? Also seems to have fragile mathematical foundations.
# Try to see if can apply transfer learning, i.e. a similar problem has already been tackled in a related domain (i.e. speech recognition)